# Imports

In [1]:
# Import utils
import numpy as np
import pandas as pd
import copy
import time
import datetime as dt
import pickle
import json
from pathlib import Path
import joblib
from joblib import dump, load, Parallel, delayed
import os
import itertools
import contextlib
from tqdm import tqdm

# Import Weights Model
import WeightsModel3
from WeightsModel3 import PreProcessing
from WeightsModel3 import RandomForestWeightsModel
from WeightsModel3 import RollingHorizonGlobalTrainingAndSampling

# Import (Rolling Horizon) Weighted SAA models
from WeightedSAA3 import WeightedSAA
from WeightedSAA3 import RobustWeightedSAA
from WeightedSAA3 import RollingHorizonOptimization

# Experiment functions

We first define several functions for experiments over different choices for the look-ahead $\tau$, cost parameter settings $\{K,u,h,b\}$, products (SKUs) $k=1,...,M$, and periods $t=1,...,T$ of the planning horizon.

In [24]:
### Function to extract global historical demand samples, weights, and actuals for each product and test period
def prep_global_samples_and_weights(samples, weights, **kwargs):

    """
    ...
    
    """
    
    SKUs = kwargs.get('SKUs', range(1,460+1))
    ts = kwargs.get('ts', range(1,13+1))
    
    # Samples     
    samples_ = {}
    for t in ts:
        samples_[t] = samples[t]['y_train']

    # Weights   
    weights_ = {}
    for SKU in SKUs:
        weights_[SKU] = {}
        for t in ts:
            weights_[SKU][t] = weights[t][samples[t]['id_test'].SKU==SKU].flatten()

    # Actuals
    actuals_ = {}
    for SKU in SKUs:
        actuals_[SKU] = {}
        for t in ts:
            actuals_[SKU][t] = samples[t]['y_test'][samples[t]['id_test'].SKU==SKU].flatten()

    return samples_, weights_, actuals_

In [25]:
### Function to extract local historical demand samples, weights, and actuals for each product and test period
def prep_local_samples_and_weights(samples, weights, **kwargs):

    """
    ...
    
    """
    
    SKUs = kwargs.get('SKUs', range(1,460+1))
    ts = kwargs.get('ts', range(1,13+1))
    
    # Samples 
    samples_ = {}
    for SKU in SKUs:
        samples_[SKU] = {}
        for t in ts:
            samples_[SKU][t] = samples[SKU][t]['y_train']

    # Weights   
    weights_ = {}
    for SKU in SKUs:
        weights_[SKU] = {}
        for t in ts:
            weights_[SKU][t] = weights[SKU][t].flatten()

    # Actuals
    actuals_ = {}
    for SKU in SKUs:
        actuals_[SKU] = {}
        for t in ts:
            actuals_[SKU][t] = samples[SKU][t]['y_test'].flatten()

    return samples_, weights_, actuals_

In [26]:
### Function to run an experiment over a list of given cost parameter settings and the specified model
def run_experiment(wsaamodel, cost_params, samples, actuals, weights=None, epsilons=None, print_progress=False,
                   path_to_save=None, name_to_save=None, return_results=True, **kwargs):
    
    """
    ...
    
    """
    
    # Raise error if cost_params is not a list of dict(s)
    if not type(cost_params)==list:
        raise ValueError('Argument cost_params has to be a list of at least one dict with keys {K, u, h, b}')  
            
    # Timer
    st_exec, st_cpu = time.time(), time.process_time()

    # Status
    if print_progress and 'SKU' in kwargs: print('SKU:', kwargs['SKU'])
    
    # Initialize
    ropt, results = RollingHorizonOptimization(), pd.DataFrame()

    # For each cost param setting
    for cost_params_ in cost_params:

        # Print progress
        if print_progress: print('...cost param setting:', cost_params_)

        # Apply (Weighted) SAA  model
        wsaamodel.set_params(**{**kwargs, **cost_params_})
        result = ropt.run(wsaamodel, samples, actuals, weights, epsilons)

        # Store results
        meta = pd.DataFrame({'CR': cost_params_['CR'], **kwargs}, index=list(range(len(samples))))
        results = pd.concat([results, pd.concat([meta, result], axis=1)], axis=0)

    # Save result as csv file
    if not path_to_save is None and not name_to_save is None:
        results.to_csv(path_or_buf=(path_to_save+'/'+name_to_save+'_SKU'+str(kwargs.get('SKU', None))+
                                    '_tau'+str(kwargs.get('tau', None))+'.csv'), sep=',', index=False)

    # Timer
    exec_time_sec, cpu_time_sec = time.time() - st_exec, time.process_time() - st_cpu
    
    # Status
    if print_progress: print('>>>> Done:', str(np.around(exec_time_sec/60,1)), 'minutes')

    # Return  
    return results if return_results else {'SKU': kwargs.get('SKU', None), 'exec_time_sec': exec_time_sec, 'cpu_time_sec': cpu_time_sec}

In [27]:
### Function to aggregate all results
def aggregateResults(taus, SKUs, path_to_save, name_to_save):
    
    results = pd.DataFrame()
    
    # For each look-ahead tau=0,...,4
    for tau in taus:
        # For each product (SKU) k=1,...,M
        for SKU in SKUs:
            file_name = path_to_save+'/'+name_to_save+'_SKU'+str(SKU)+'_tau'+str(tau)+'.csv'
            # Check if results exist   
            if os.path.exists(file_name):
                results = pd.concat([results, pd.read_csv(file_name)])

    return(results)

In [28]:
### Context manager (Credits: 'https://stackoverflow.com/questions/24983493/tracking-progress-of-joblib-parallel-execution')
@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

# Paths and parameters

In [29]:
# Set folder names as global variables
os.chdir('/home/fesc/MM/')
global PATH_DATA, PATH_PARAMS, PATH_KERNELS, PATH_SAMPLES, PATH_RESULTS

PATH_DATA = '/home/fesc/MM/Data'
PATH_PARAMS  = '/home/fesc/MM/Data/Params'
PATH_WEIGHTSMODEL = '/home/fesc/MM/Data/WeightsModel'
PATH_SAMPLES = '/home/fesc/MM/Data/Samples'
PATH_RESULTS = '/home/fesc/MM/Data/Results'

In [36]:
# Define experiment paramaters
experiment_params = {
            
    # Cost param settings
    'cost_params': [

        {'CR': 0.50, 'K': 100, 'u': 0.5, 'h': 1, 'b': 1},
        {'CR': 0.75, 'K': 100, 'u': 0.5, 'h': 1, 'b': 3},
        {'CR': 0.90, 'K': 100, 'u': 0.5, 'h': 1, 'b': 9}

    ],

    # Gurobi meta params
    'LogToConsole': 0, 
    'Threads': 1, 
    'NonConvex': 2, 
    'PSDTol': 1e-3, # 0.1%
    'MIPGap': 1e-3, # 0.1%
    'NumericFocus': 0, 
    'obj_improvement': 1e-3, # 0.1%
    'obj_timeout_sec': 3*60, # 3 min
    'obj_timeout_max_sec': 10*60, # 10 min

    # Program meta params
    'path_to_save': '',
    'name_to_save': '',
    'print_progress': False,
    'return_results': False

}

In [30]:
# Time period and SKU ranges
T = 13                  # Planning horizon T
ts = range(1,13+1)      # Periods t=1,...,T of the planning horizon
taus = range(0,4+1)     # Look-aheads tau=0,...,4 to use
SKUs = range(1,460+1)   # Products (SKUs) k=1,...,M

# Train/test split (first timePeriods of testing horizon)
test_start = 114

# Global Models

The two global models (using 'Global Training and Sampling') are **Rolling Horizon Global Weighted SAA (GwSAA)**, which is our model, and **Rolling Horizon Global Robust Weighted SAA (GwSAA-R)**, which is the analogous model with robust extension.

Given product $k$, period $t$, and look-ahead $\tau$, both models apply Weighted SAA over the 'global' distribution $\{\{w_{j,t,\tau}^{\,i}(x_{k,t}^{\,i}),(d_{j,t}^{\,i},...,d_{j,t+\tau}^{\,i})\}_{i=1}^{N_{j,t,\tau}}\}_{j=1}^{M}$, with weight functions $w_{j,t,\tau}(\,\cdot\,)$ trained (once for all products) on data $S_{t,\tau}^{\,\text{Global}}=\{\{(x_{j,t}^{\,i},d_{j,t}^{\,i},...,d_{j,t+\tau}^{\,i})\}_{i=1}^{N_{j,t,\tau}}\}_{j=1}^{M}$.

## Data Preprocessing

We first load and pre-process the data. This includes reshaping demand time series into $(\tau+1)$-periods rolling look-ahead horizon sequences.

- **ID_Data** (pd.DataFrame) stores identifiers (in particular the product (SKU) identifier and the timePeriod (sale_yearweek) identifier)
- **X_Data** (pd.DataFrame) is the 'feature matrix', i.e., each row is a feature vector $x_{j,n}$ where n is the number of training observations (rows) in the data
- **Y_Data** (pd.DataFrame) is the demand data $d_{j,n}$ (a times series per product)
- **X_Data_Columns** (pd.DataFrame) provides 'selectors' for local vs. global feature sets

In [ ]:
# Weights model names
weightsmodel_cv_name = 'cv_rfwm_global'
weightsmodel_name = 'rfwm_global'

In [ ]:
# Read data
ID_Data = pd.read_csv(PATH_DATA+'/ID_Data.csv')
X_Data = pd.read_csv(PATH_DATA+'/X_Data.csv')
X_Data_Columns = pd.read_csv(PATH_DATA+'/X_Data_Columns.csv')
Y_Data = pd.read_csv(PATH_DATA+'/Y_Data.csv')

In [ ]:
# Select features
X_Data_Columns = X_Data_Columns.loc[X_Data_Columns.Global == 'YES']
X_Data = X_Data[X_Data_Columns.Feature.values]

In [ ]:
# Ensure data is sorted by SKU and sale_yearweek for preprocessing
data = pd.concat([ID_Data, X_Data, Y_Data], axis=1).sort_values(by=['SKU', 'sale_yearweek']).reset_index(drop=True)
ID_Data = data[ID_Data.columns]
X_Data = data[X_Data.columns]
Y_Data = data[Y_Data.columns]

# Create multi-period demand vectors
data = pd.concat([ID_Data, Y_Data], axis=1)
Y = {}
for tau in taus:
    Y['Y'+str(tau)] = data.groupby(['SKU']).shift(-tau)['Y']
Y_Data = pd.DataFrame(Y)

## Weights model

The weights models - and thus the data used, weight functions, and weights per sample - are the same for the two global models **GwSAA** and **GwSAA-R**. First, we tune the hyper parameters of the random forest weights model for each given look-ahead $\tau$ (as for each look-ahead $\tau$ we have a different response for the multi-output random forest regressor). Second, we fit all weight functions (for each look-ahead $\tau=0,...,4$ and over periods $t=1,...,T$) and generate all weights (for each look-ahead $\tau=0,...,4$, over periods $t=1,...,T$, and for each product (SKU) $k=1,...,M$).

### Tune weights model

To tune the hyper parameters of the global random forest weights model, we use 3-fold rolling timeseries cross-validation on the training data and perform random search with 100 iterations over the specified hyper parameter search grid.

In [ ]:
# Set parameters
model_params = {
    'oob_score': True,
    'random_state': 12345,
    'n_jobs': 4,
    'verbose': 0
}

hyper_params_grid = {
    'n_estimators': [1000],
    'max_depth': [None],
    'min_samples_split': [x for x in range(20, 1000, 20)],  
    'min_samples_leaf': [x for x in range(10, 1000, 10)],  
    'max_features': [x for x in range(8, 256, 8)],   
    'max_leaf_nodes': [None],
    'min_impurity_decrease': [0.0],
    'bootstrap': [True],
    'max_samples': [0.75, 0.80, 0.85, 0.90, 0.95, 1.00]
}    


tuning_params = {     
    'n_iter': 100,
    'scoring': {'MSE': 'neg_mean_squared_error'},
    'return_train_score': True,
    'refit': 'MSE',
    'random_state': 12345,
    'n_jobs': 8,
    'verbose': 2
}    

random_search = True
print_status = True

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Initialize preprocessing module
    pp = PreProcessing()
        
    # Select and reshape training and test data
    args = {'train': (ID_Data.sale_yearweek < test_start), 
            'timePeriods': ID_Data.loc[(ID_Data.sale_yearweek < test_start)].sale_yearweek, 
            'maxTimePeriod': test_start-1, 'tau': tau}
    
    id_train = pp.train_test_split(ID_Data, **args)
    X_train = pp.train_test_split(X_Data, **args, to_array=True)
    y_train = pp.train_test_split(Y_Data, **args, rolling_horizon=[l for l in range(0,tau+1)], to_array=True)
   
    # Initialize
    weightsmodel = RandomForestWeightsModel(model_params)

    # CV search
    cv_folds = pp.split_timeseries_cv(n_splits=3, timePeriods=id_train.sale_yearweek)
    cv_results = weightsmodel.tune(X_train, y_train, cv_folds, hyper_params_grid, tuning_params, random_search, print_status)
    weightsmodel.save_cv_result(path=PATH_WEIGHTSMODEL+'/'+weightsmodel_cv_name+'_tau'+str(tau)+'.joblib')

### Fit weights model and generate weights

We now fit the global random forest weights model (i.e., the weight functions) for each $\tau=0,...,4$ and over periods $t=1,...,T$. This is done across all products at once (global training). Then, for each $\tau=0,...,4$ and over periods $t=1,...,T$, we generate for each product (SKU) $k=1,...,M$ the weights given the test feature $x_{k,t}$. This is done *jointly* across products (by using $x_{t}=(x_{1,t},...,x_{M,t})^{\top}$) for computational efficiency - the weights for each individual product can be extracted afterwards.

In [ ]:
# Set parameters
model_params = {
    'n_jobs': 32,
    'verbose': 0
}

print_status = True

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Initialize
    samples, weightfunctions, weightfunctions_times, weights, weights_times = {}, {}, {}, {}, {}
        
    # For each period t=1,...,T
    for t in ts:

        # Adjust look-ahead tau to account for end of horizon
        tau_ = min(tau,T-t)
        
        # Status
        print('#### Look-ahead tau='+str(tau)+' (tau\'='+str(tau_)+'), period t='+str(t)+'...')
        start_time = dt.datetime.now().replace(microsecond=0)
                
        # Generate samples, fit weight functions, and generate weights (based on tuned weights model)
        weightsmodel = RandomForestWeightsModel()
        weightsmodel.load_cv_result(path=PATH_WEIGHTSMODEL+'/'+weightsmodel_cv_name+'_tau'+str(tau_)+'.joblib')
        res = weightsmodel.training_and_sampling(ID_Data, X_Data, Y_Data, tau=tau_, timePeriods=ID_Data.sale_yearweek,
                                                 timePeriodsTestStart=test_start+t-1, model_params=model_params)
        samples[t], weightfunctions[t], weightfunctions_times[t], weights[t], weights_times[t] = res
        
        # Status
        print('...done in', dt.datetime.now().replace(microsecond=0) - start_time)    
        
    # Save
    _ = joblib.dump(samples, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_samples_tau'+str(tau)+'.joblib')  
    _ = joblib.dump(weightfunctions, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weightfunctions_tau'+str(tau)+'.joblib')    
    _ = joblib.dump(weightfunctions_times, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weightfunctions_times_tau'+str(tau)+'.joblib')    
    _ = joblib.dump(weights, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights_tau'+str(tau)+'.joblib')    
    _ = joblib.dump(weights_times, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights_times_tau'+str(tau)+'.joblib')    

## Optimization

Description ...

### Rolling Horizon Global Weighted SAA (GwSAA)

The code below runs an experiment for all given products (SKUs) $k=1,...,M$ over a test planning horizon $t=1,...,T$ with $T=13$ for three different cost parameter settings $\{K, u, h, b\}$ that vary the critical ratio ($CR=\frac{b}{b+h}$) of holding and backlogging yielding
- $CR=0.50$: $\{K=100, u=0.5, h=1, b=1\}$
- $CR=0.75$: $\{K=100, u=0.5, h=1, b=3\}$
- $CR=0.90$: $\{K=100, u=0.5, h=1, b=9\}$

In [ ]:
# Define experiment paramaters
experiment_params = {
            
    # Cost param settings
    'cost_params': [

        {'CR': 0.50, 'K': 100, 'u': 0.5, 'h': 1, 'b': 1},
        {'CR': 0.75, 'K': 100, 'u': 0.5, 'h': 1, 'b': 3},
        {'CR': 0.90, 'K': 100, 'u': 0.5, 'h': 1, 'b': 9}

    ],

    # Gurobi meta params
    'LogToConsole': 0, 
    'Threads': 1, 
    'NonConvex': 2, 
    'PSDTol': 1e-3, # 0.1%
    'MIPGap': 1e-3, # 0.1%
    'NumericFocus': 0, 
    'obj_improvement': 1e-3, # 0.1%
    'obj_timeout_sec': 3*60, # 3 min
    'obj_timeout_max_sec': 10*60, # 10 min

    # Program meta params
    'path_to_save': PATH_RESULTS+'/GwSAA_NEW',
    'name_to_save': 'GwSAA_NEW',
    'print_progress': False,
    'return_results': False

}

In [ ]:
# Set path
if not os.path.exists(experiment_params['path_to_save']): os.mkdir(experiment_params['path_to_save'])

# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Prepare data
    samples = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_samples_tau'+str(tau)+'.joblib')
    weights = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights_tau'+str(tau)+'.joblib')

    samples, weights, actuals = prep_global_samples_and_weights(samples, weights, SKUs=SKUs, ts=ts)
    
    # For each product (SKU) k=1,...,M
    with tqdm_joblib(tqdm(desc='Progress', total=len(SKUs))) as progress_bar:
        resultslog = Parallel(n_jobs=32)(delayed(run_experiment)(tau=tau, SKU=SKU, wsaamodel=WeightedSAA(), 
                                                                 samples=samples, weights=weights[SKU], actuals=actuals[SKU], 
                                                                 **experiment_params) for SKU in SKUs)

In [ ]:
# Aggregate results
results = aggregateResults(taus, SKUs, experiment_params['path_to_save'], experiment_params['name_to_save'])
results

In [ ]:
# Save aggregate results
file_name = experiment_params['path_to_save']+'/'+experiment_params['name_to_save']+'_results.csv'
results.to_csv(file_name, sep=',', index=False)

In [ ]:
# Summarize results
results_summary = results.groupby(['SKU', 'CR', 'tau']).agg({
    'MIPGap': lambda x: x.iloc[0],
    'NumericFocus': lambda x: x.iloc[0],
    'obj_improvement': lambda x: x.iloc[0],
    'obj_timeout_sec': lambda x: x.iloc[0],
    'obj_timeout_max_sec': lambda x: x.iloc[0],
    'K': lambda x: x.iloc[0],
    'u': lambda x: x.iloc[0],
    'h': lambda x: x.iloc[0],
    'b': lambda x: x.iloc[0],
    'I': np.mean,
    'q': np.mean,
    'I_q': np.mean,
    'y': np.mean,
    'I_q_y': np.mean,
    'c_o': sum,
    'c_s': sum,
    'cost': sum,
    'defaulted': sum,
    'solutions': lambda x: sum(x>0),
    'gap': np.mean,
    'exec_time_sec': np.mean,
    'cpu_time_sec': np.mean
}).reset_index()
results_summary

In [ ]:
# Save summarized results
file_name = experiment_params['path_to_save']+'/'+experiment_params['name_to_save']+'_results_summary.csv'
results_summary.to_csv(file_name, sep=',', index=False)

In [ ]:
# Evaluate results
results_evaluation = results_summary.groupby(['SKU', 'CR']).agg({
    'cost': min
}).reset_index()
results_evaluation

In [ ]:
# Save evaluated results
file_name = experiment_params['path_to_save']+'/'+experiment_params['name_to_save']+'_results_evaluation.csv'
results_evaluation.to_csv(file_name, sep=',', index=False)

### Rolling Horizon Global Robust Weighted SAA (GwSAA-R)

Description ...

# Local Models

The two local models (using 'Local Training and Sampling') are **Rolling Horizon Local Weighted SAA (wSAA)**, and **Rolling Horizon Local Robust Weighted SAA (wSAA-R)**, which is the analogous model with robust extension.

Given product $k$, period $t$, and look-ahead $\tau$, both models apply Weighted SAA over the 'local' distribution $\{w_{k,t,\tau}^{\,i}(x_{k,t}^{\,i}),(d_{k,t}^{\,i},...,d_{k,t+\tau}^{\,i})\}_{i=1}^{N_{k,t,\tau}}$, with weight functions $w_{k,t,\tau}(\,\cdot\,)$ trained on data $S_{k,t,\tau}^{\,\text{Local}}=\{(x_{k,t}^{\,i},d_{k,t}^{\,i},...,d_{k,t+\tau}^{\,i})\}_{i=1}^{N_{k,t,\tau}}$ for each product $k=1,...,M$ separately.

## Data Preprocessing

We first load and pre-process the data. This includes reshaping demand time series into $(\tau+1)$-periods rolling look-ahead horizon sequences.

- **ID_Data** (pd.DataFrame) stores identifiers (in particular the product (SKU) identifier and the timePeriod (sale_yearweek) identifier)
- **X_Data** (pd.DataFrame) is the 'feature matrix', i.e., each row is a feature vector $x_{j,n}$ where n is the number of training observations (rows) in the data
- **Y_Data** (pd.DataFrame) is the demand data $d_{j,n}$ (a times series per product)
- **X_Data_Columns** (pd.DataFrame) provides 'selectors' for local vs. global feature sets

In [31]:
# Weights model names
weightsmodel_cv_name = 'cv_rfwm_local'
weightsmodel_name = 'rfwm_local'

In [32]:
# Read data
ID_Data = pd.read_csv(PATH_DATA+'/ID_Data.csv')
X_Data = pd.read_csv(PATH_DATA+'/X_Data.csv')
X_Data_Columns = pd.read_csv(PATH_DATA+'/X_Data_Columns.csv')
Y_Data = pd.read_csv(PATH_DATA+'/Y_Data.csv')

In [33]:
# Select features
X_Data_Columns = X_Data_Columns.loc[X_Data_Columns.Local == 'YES']
X_Data = X_Data[X_Data_Columns.Feature.values]

In [34]:
# Ensure data is sorted by SKU and sale_yearweek for preprocessing
data = pd.concat([ID_Data, X_Data, Y_Data], axis=1).sort_values(by=['SKU', 'sale_yearweek']).reset_index(drop=True)

ID_Data = data[ID_Data.columns]
X_Data = data[X_Data.columns]
Y_Data = data[Y_Data.columns]

In [35]:
# Create multi-period demand vectors
data = pd.concat([ID_Data, Y_Data], axis=1)
Y = {}
for tau in taus:
    Y['Y'+str(tau)] = data.groupby(['SKU']).shift(-tau)['Y']
Y_Data = pd.DataFrame(Y)

## Weights model

The weights model - and thus the data used, weight functions, and weights per sample - are the same for the two local models **wSAA** and **wSAA-R**. First, we tune the hyper parameters of the random forest weights model for each given look-ahead $\tau$ (as for each look-ahead $\tau$ we have a different response for the multi-output random forest regressor) and for each product (SKU) $k=1,...,M$ separately. Second, we fit all weight functions (for each look-ahead $\tau=0,...,4$ and over periods $t=1,...,T$) for each product (SKU) $k=1,...,M$ separately and generate all weights (for each look-ahead $\tau=0,...,4$, over periods $t=1,...,T$, and for each product (SKU) $k=1,...,M$ separatey).

### Tune weights model

To tune the hyper parameters of the local random forest weights model for each product (SKU) $k=1,...,M$, we use 3-fold rolling timeseries cross-validation on the training data and perform random search with 100 iterations over the specified hyper parameter search grid.

In [ ]:
# Set parameters to tune random forest weights kernels
model_params = {
    'oob_score': True,
    'random_state': 12345,
    'n_jobs': 1,
    'verbose': 0
}

hyper_params_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None],
    'min_samples_split': [x for x in range(2, 20, 1)],  
    'min_samples_leaf': [x for x in range(2, 10, 1)],  
    'max_features': [x for x in range(8, 256, 8)],   
    'max_leaf_nodes': [None],
    'min_impurity_decrease': [0.0],
    'bootstrap': [True],
    'max_samples': [0.75, 0.80, 0.85, 0.90, 0.95, 1.00]
}    


tuning_params = {     
    'n_iter': 100,
    'scoring': {'MSE': 'neg_mean_squared_error'},
    'return_train_score': True,
    'refit': 'MSE',
    'random_state': 12345,
    'n_jobs': 32,
    'verbose': 0
}    

random_search = True
print_status = False

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Status
    print('Look-ahead tau='+str(tau)+'...')
    start_time = dt.datetime.now().replace(microsecond=0)
    
    # Initialize
    cv_results = {}
    
    # For each product (SKU) k=1,...,M
    for SKU in SKUs:

        # Initialize preprocessing module
        pp = PreProcessing()

        # Select and reshape training and test data
        args = {'train': (ID_Data.SKU == SKU) & (ID_Data.sale_yearweek < test_start), 
                'timePeriods': ID_Data.loc[(ID_Data.SKU == SKU) & (ID_Data.sale_yearweek < test_start)].sale_yearweek, 
                'maxTimePeriod': test_start-1, 'tau': tau}

        id_train = pp.train_test_split(ID_Data, **args)
        X_train = pp.train_test_split(X_Data, **args, to_array=True)
        y_train = pp.train_test_split(Y_Data, **args, rolling_horizon=[l for l in range(0,tau+1)], to_array=True)

        # Initialize
        weightsmodel = RandomForestWeightsModel(model_params)

        # CV search
        cv_folds = pp.split_timeseries_cv(n_splits=3, timePeriods=id_train.sale_yearweek)
        cv_results[SKU] = weightsmodel.tune(X_train, y_train, cv_folds, hyper_params_grid, 
                                            tuning_params, random_search, print_status)
        
        # Status
        print('SKU '+str(SKU)+' of '+str(len(SKUs))+' in', dt.datetime.now().replace(microsecond=0) - start_time, end='\r', flush=True)

    # Save
    _ = joblib.dump(cv_results, PATH_WEIGHTSMODEL+'/'+weightsmodel_cv_name+'_tau'+str(tau)+'.joblib')
    print('')

### Fit weights model and generate weights

We now fit a local random forest weights model (i.e., the weight functions) for each $\tau=0,...,4$, period $t=1,...,T$, and product (SKU) $k=1,...,M$ separately (local training). Then, for each $\tau=0,...,4$, period $t=1,...,T$, and product (SKU) $k=1,...,M$ separately, we generate the weights given the test feature $x_{k,t}$. This is done *separately* for each product (SKU) $k=1,...,M$.

In [9]:
# Set parameters
model_params = {
    'n_jobs': 32,
    'verbose': 0
}

print_status = True

In [10]:
# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Status
    print('Look-ahead tau='+str(tau)+'...')
    start_time = dt.datetime.now().replace(microsecond=0)
    
    # Initialize
    samples, weightfunctions, weightfunctions_times, weights, weights_times = {}, {}, {}, {}, {}
    
    # For each product (SKU) k=1,...,M
    for SKU in SKUs:
        
        # Initialize
        samples[SKU], weightfunctions[SKU], weightfunctions_times[SKU], weights[SKU], weights_times[SKU] = {}, {}, {}, {}, {}
        
        # For each period t=1,...,T
        for t in ts:
        
            # Adjust look-ahead tau to account for end of horizon
            tau_ = min(tau,T-t)

            # Generate samples, fit weight functions, and generate weights (based on tuned weights model)
            weightsmodel = RandomForestWeightsModel()
            weightsmodel.load_cv_result(path=PATH_WEIGHTSMODEL+'/'+weightsmodel_cv_name+'_tau'+str(tau_)+'.joblib', SKU=SKU)
            res = weightsmodel.training_and_sampling(ID_Data.loc[ID_Data.SKU==SKU], X_Data.loc[ID_Data.SKU==SKU], Y_Data.loc[ID_Data.SKU==SKU], 
                                                     tau=tau_, timePeriods=ID_Data.loc[ID_Data.SKU==SKU].sale_yearweek, 
                                                     timePeriodsTestStart=test_start+t-1, model_params=model_params)
            samples[SKU][t], weightfunctions[SKU][t], weightfunctions_times[SKU][t], weights[SKU][t], weights_times[SKU][t] = res

        # Status
        print('SKU '+str(SKU)+' of '+str(len(SKUs))+' in', dt.datetime.now().replace(microsecond=0) - start_time, end='\r', flush=True)
        
    # Save
    _ = joblib.dump(samples, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_samples_tau'+str(tau)+'.joblib')  
    _ = joblib.dump(weightfunctions, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weightfunctions_tau'+str(tau)+'.joblib')    
    _ = joblib.dump(weightfunctions_times, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weightfunctions_times_tau'+str(tau)+'.joblib')    
    _ = joblib.dump(weights, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights_tau'+str(tau)+'.joblib')    
    _ = joblib.dump(weights_times, PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights_times_tau'+str(tau)+'.joblib')
    print('')

Look-ahead tau=0...
SKU 460 of 460 in 0:43:49
Look-ahead tau=1...
SKU 460 of 460 in 0:45:11
Look-ahead tau=2...
SKU 460 of 460 in 0:42:45
Look-ahead tau=3...
SKU 460 of 460 in 0:39:13
Look-ahead tau=4...
SKU 460 of 460 in 0:42:28


### Rolling Horizon Local Weighted SAA (wSAA)

The code below runs an experiment for all given products (SKUs) $k=1,...,M$ over a test planning horizon $t=1,...,T$ with $T=13$ for three different cost parameter settings $\{K, u, h, b\}$ that vary the critical ratio ($CR=\frac{b}{b+h}$) of holding and backlogging yielding
- $CR=0.50$: $\{K=100, u=0.5, h=1, b=1\}$
- $CR=0.75$: $\{K=100, u=0.5, h=1, b=3\}$
- $CR=0.90$: $\{K=100, u=0.5, h=1, b=9\}$

In [ ]:
# Define experiment paramaters
experiment_params = {
            
    # Cost param settings
    'cost_params': [

        {'CR': 0.50, 'K': 100, 'u': 0.5, 'h': 1, 'b': 1},
        {'CR': 0.75, 'K': 100, 'u': 0.5, 'h': 1, 'b': 3},
        {'CR': 0.90, 'K': 100, 'u': 0.5, 'h': 1, 'b': 9}

    ],

    # Gurobi meta params
    'LogToConsole': 0, 
    'Threads': 1, 
    'NonConvex': 2, 
    'PSDTol': 1e-3, # 0.1%
    'MIPGap': 1e-3, # 0.1%
    'NumericFocus': 0, 
    'obj_improvement': 1e-3, # 0.1%
    'obj_timeout_sec': 3*60, # 3 min
    'obj_timeout_max_sec': 10*60, # 10 min

    # Program meta params
    'path_to_save': PATH_RESULTS+'/wSAA_NEW',
    'name_to_save': 'wSAA_NEW',
    'print_progress': False,
    'return_results': False

}

In [ ]:
# Set path
if not os.path.exists(experiment_params['path_to_save']): os.mkdir(experiment_params['path_to_save'])

# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Prepare data
    samples = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_samples_tau'+str(tau)+'.joblib')
    weights = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights_tau'+str(tau)+'.joblib')

    samples, weights, actuals = prep_local_samples_and_weights(samples, weights, SKUs=SKUs, ts=ts)
    
    # For each product (SKU) k=1,...,M
    with tqdm_joblib(tqdm(desc='Progress', total=len(SKUs))) as progress_bar:
        resultslog = Parallel(n_jobs=32)(delayed(run_experiment)(tau=tau, SKU=SKU, wsaamodel=WeightedSAA(), 
                                                                 samples=samples[SKU], weights=weights[SKU], actuals=actuals[SKU], 
                                                                 **experiment_params) for SKU in SKUs)

In [17]:
tau=3
# Prepare data
samples = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_samples_tau'+str(tau)+'.joblib')
weights = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights_tau'+str(tau)+'.joblib')

samples, weights, actuals = prep_local_samples_and_weights(samples, weights, SKUs=SKUs, ts=ts)

### Rolling Horizon Local Robust Weighted SAA (wSAA-R)

Description ...

# Baseline model: Rolling Horizon Local Weighted SAA (SAA)

Description ...

# Ex-post optimal model

Description ...

# ARCHIVE >>>

# Functions

## Data preparation

We first define a function that prepares the data needed for an experiment (depending on the model/approach). 

If no sampling strategy is provided via the optional argument 'sampling', no weights are retrieved, else 'global' or 'local' weights are retrieved and historical demands are prepared for 'global' or 'local' sampling, respectively. 
    
If the optional argument 'e' is provided, the function additionally outputs 'epsilon' which is the uncertainty set threshold for robust optimization.

In [ ]:
## Prepare the data to a run an experiment over the full planning horizon
def prep_data(SKU, tau, T, sale_yearweek, path_data, path_samples, **kwargs):

    """

    This function prepares the data needed for (weighted, robust) optimization. If no sampling strategy is
    provided via the optional argument 'sampling', no weights are retrieved, else 'global' or 'local' weights
    are retrieved and historical demands are prepared for 'global' or 'local' sampling, respectively. If the
    optional argument 'e' is provided, the function additionally outputs 'epsilon' which is the uncertainty
    set threshold for robust optimization.

    Arguments:

        SKU: product (SKU) identifier
        tau: length of rolling look-ahead horizon
        T: Length T of the test horizon
        sale_yearweek: Last sale_yearweek of training data
        path_data: path of data
        path_samples: path of samples

    Optional arguments: 

        sampling: Sampling strategy (either 'global', 'local'), with
            - 'global': uses weights generated with global training
            - 'local': uses weights generated with local training
        e: Robust uncertainty set threshold multiplier, with
            - int: uses e as multiplier for product's in sample standard deviation as the uncertainty set threshold 

    Output:

        y: demand data - np.array of shape (n_samples, n_periods)
        ids_train: list of selector series (True/False of length n_samples) - list with lengths of the test horizon
        ids_test: list of selector series (True/False of length n_samples) - list with lengths of the test horizon

        weights (optional): list of weights (flat np.array of length ids_train of t'th test period) - list 
        with length of test horizon
        epsilons (optional): list of epsilons - list with length of the test horizon

    """

    # Demand samples
    robj = pyreadr.read_r(path_data+'/Y_Data_mv_NEW.RData')
    y_samples = np.array(robj['Y_Data_mv'])

    # IDs of local demand samples
    robj = pyreadr.read_r(path_data+'/ID_Data_NEW.RData')
    ID_samples = robj['ID_Data']

    # IDs of local demand samples
    robj = pyreadr.read_r(path_samples+'/SKU'+str(SKU)+'/Static/TmpFiles'+
                          str(tau)+'/ID_samples_k.RDS')
    ID_samples_SKU = robj[None]

    # If sampling strategy is provided
    if 'sampling' in kwargs:

        # Weights
        with open(path_samples+'/SKU'+str(SKU)+'/Static/Weights'+
                  str(tau)+'/weights_'+kwargs['sampling']+'_ij.p', 'rb') as f:
            weighty_ij = pickle.load(f)
        del f

        # Demand samples for global sampling
        if kwargs['sampling'] == 'global':
            y = y_samples

        # Demand samples for local sampling
        if kwargs['sampling'] == 'local':
            y = y_samples[ID_samples.SKU_API == ID_samples_SKU.SKU_API[0]]

    # Default: local demand samples
    else:
        y = y_samples[ID_samples.SKU_API == ID_samples_SKU.SKU_API[0]]


    # Reshape data for each t=1...T (i.e., each period of the test horizon)
    ids_train = []
    ids_test = []

    weights = [] if 'sampling' in kwargs else None
    epsilons = [] if 'e' in kwargs else None

    # Iterate over t
    for t in range(T):

        # If sampling strategy is provided
        if 'sampling' in kwargs:

            # IDs of demand samples for global sampling
            if kwargs['sampling'] == 'global':
                ids_train = ids_train + [ID_samples.sale_yearweek < sale_yearweek+t]

            # IDs of demand samples for local sampling
            if kwargs['sampling'] == 'local':
                ids_train = ids_train + [(ID_samples.SKU_API == ID_samples_SKU.SKU_API[0]) &
                                         (ID_samples.sale_yearweek < sale_yearweek+t)]                   

            # Weights for global/local
            weights = weights + [weighty_ij[t+1]]

        # Default: IDs of demand samples for local sampling
        else:
            ids_train = ids_train + [(ID_samples.SKU_API == ID_samples_SKU.SKU_API[0]) &
                                         (ID_samples.sale_yearweek < sale_yearweek+t)]



        # IDs of demand samples for testing 
        ids_test = ids_test + [(ID_samples.SKU_API == ID_samples_SKU.SKU_API[0]) &
                                         (ID_samples.sale_yearweek == sale_yearweek+t)]


        # If e is provided, calculate robust optimization parameter epsilon
        if 'e' in kwargs:
            epsilons = epsilons + [kwargs['e']*np.std(y_samples[(ID_samples.SKU_API == ID_samples_SKU.SKU_API[0]) &
                                                                (ID_samples.sale_yearweek < sale_yearweek+t),0])]


    # Return
    return y, ids_train, ids_test, weights, epsilons

## Experiment wrapper

We now define a 'wrapper' function that iterates the experiment for a given SKU over differen cost parameter settins and lengths of the rolling look-ahead horizon tau.

If the parameter sampling is provided (either 'global' or 'local), the function uses the specified sampling strategy. Else, SAA is performed. If the multiplier 'e' for the uncertainty set threshold epsilon is provided, the function performs the robust extension.
 
Where: epsilon[t] = e *  in-sample standard deviation of the current product (SKU).

The function prepares and calls the experiment over t=1...T for each cost paramater setting and look-ahead horizon tau and then summarises the results including performance and performance meta inormation. It also saves the results in CSV format to the specified path and the function can also be used in parallel processing environments.

In [ ]:
def run_experiments(SKU, **kwargs):
    
    """
    
    Description ...
    
    
    Arguments:
    
        SKU: product (SKU) identifier
        sale_yearweek: Last sale_yearweek of training data
        T: Length T of the test horizon
        tau: List of lengths of rolling look-ahead horizons
        cost_params: dictionary/dictionary of dictionaries of cost parameters {'CR', 'K', 'u', 'h', 'b'}
        gurobi_params: dictionary of gurobi meta params {'LogToConsole', 'Threads', 'NonConvex' 
                                                         'PSDTol', 'MIPGap', 'NumericFocus',
                                                         'obj_improvement', obj_timeout_sec'}
        path: directory where results should be saved
        model_name: model name for the file to save results
        
    Optional arguments:
    
        sampling: sampling strategy (either 'global' or 'local'); performs SAA if not provided
        e: robust uncertainty set threshold multiplier; performs no robust extension if not provided
    

    """
  
    st_exec = time.time()
    st_cpu = time.process_time()
    
    # Print progress
    if kwargs['print_progress']: 
        print('SKU:', SKU)
    
    # Initialize
    rhopt = RollingHorizonOptimization()
    results = pd.DataFrame()
   
    # For each cost param setting
    for cost_params in kwargs['cost_params'].values():
        
        # Print progress
        if kwargs['print_progress']: 
            print('... cost param setting:', cost_params)
    
        # For each rolling look-ahead horizon
        for tau in kwargs['tau']:
            
            # Print progress
            if kwargs['print_progress']: 
                print('...... look-ahead horizon:', tau)
    
            ## Weighted (Robust) SAA
            if 'sampling' in kwargs:
    
                ## Weighted Robust SAA  
                if 'e' in kwargs:

                    # Prepare data
                    data = prep_data(SKU, tau, kwargs['T'], kwargs['sale_yearweek'], PATH_DATA, PATH_SAMPLES, sampling=kwargs['sampling'], e=kwargs['e'])
                    y, ids_train, ids_test, weights, epsilons = data
                    
                    # Create empty model
                    wsaamodel = RobustWeightedSAA(**kwargs['gurobi_params'])

                    # Run rolling horizon model over t=1...T
                    result = rhopt.run(y, ids_train, ids_test, tau, wsaamodel, weights=weights, epsilons=epsilons, **cost_params)

                ## Weighted SAA
                else: 
                    
                    # Prepare data
                    data = prep_data(SKU, tau, kwargs['T'], kwargs['sale_yearweek'], PATH_DATA, PATH_SAMPLES, sampling=kwargs['sampling'])
                    y, ids_train, ids_test, weights, _ = data
                    
                    # Create empty model
                    wsaamodel = WeightedSAA(**kwargs['gurobi_params'])

                    # Run rolling horizon model over t=1...T
                    result = rhopt.run(y, ids_train, ids_test, tau, wsaamodel, weights=weights, **cost_params)


            ## SAA
            else:
                
                # Prepare data
                data = prep_data(SKU, tau, kwargs['T'], kwargs['sale_yearweek'], PATH_DATA, PATH_SAMPLES)
                y, ids_train, ids_test, _, _ = data

                # Create empty model
                wsaamodel = WeightedSAA(**kwargs['gurobi_params'])

                # Run rolling horizon model over t=1...T
                result = rhopt.run(y, ids_train, ids_test, tau, wsaamodel, **cost_params)

            
            ## ToDo: ExPost
            
            # Store result
            meta = pd.DataFrame({

                'SKU': np.repeat(SKU,kwargs['T']),
                'n_periods': np.repeat(kwargs['T'],kwargs['T']),
                'tau': np.repeat(tau,kwargs['T']),
                'CR': np.repeat(cost_params['CR'],kwargs['T']),
                'LogToConsole': np.repeat(kwargs['gurobi_params']['LogToConsole'],kwargs['T']),
                'Threads': np.repeat(kwargs['gurobi_params']['Threads'],kwargs['T']),
                'NonConvex': np.repeat(kwargs['gurobi_params']['NonConvex'],kwargs['T']),
                'PSDTol': np.repeat(kwargs['gurobi_params']['PSDTol'],kwargs['T']),
                'MIPGap': np.repeat(kwargs['gurobi_params']['MIPGap'],kwargs['T']),
                'NumericFocus': np.repeat(kwargs['gurobi_params']['NumericFocus'],kwargs['T']),
                'obj_improvement': np.repeat(kwargs['gurobi_params']['obj_improvement'],kwargs['T']),
                'obj_timeout_sec': np.repeat(kwargs['gurobi_params']['obj_timeout_sec'],kwargs['T']),
                'e': np.repeat(kwargs['e'],kwargs['T']) if 'e' in kwargs else np.repeat(0,kwargs['T']),
                'epsilon': [epsilon for epsilon in epsilons] if 'e' in kwargs else np.repeat(0,kwargs['T'])
            })

            result = pd.concat([meta, result], axis=1)

            # Store
            if not results.empty:
                results = results.append(result)   
            else:
                results = pd.DataFrame(result) 

    # Save result
    save_log = results.to_csv(
        path_or_buf=kwargs['path']+'/'+kwargs['model_name']+'_SKU'+str(SKU)+(('_e'+str(kwargs['e'])) if 'e' in kwargs else '')+'.csv', 
        sep=',', index=False
    )
    
    
    # Time
    exec_time_sec = time.time() - st_exec
    cpu_time_sec = time.process_time() - st_cpu
    
    # Print progress
    if kwargs['print_progress']: 
        print('>>>> Done:',str(np.around(exec_time_sec/60,1)), 'minutes')

    
    # Returns results 
    if (kwargs['return_results'] if 'return_results' in kwargs else False):
        return results
    
    # Returns a log
    else:
        return  {'SKU': SKU, 'exec_time_sec': exec_time_sec, 'cpu_time_sec': cpu_time_sec}

**Context Manager**

This is a context manager for parellel execution with the purpose of reporting progress. 

Credits: https://stackoverflow.com/questions/24983493/tracking-progress-of-joblib-parallel-execution

In [ ]:
@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

# Experiments

In [ ]:
# Set folder names as global variables
os.chdir('/home/fesc/')
global PATH_DATA, PATH_PARAMS, PATH_SAMPLES, PATH_RESULTS

PATH_DATA = '/home/fesc/MM/Data'
PATH_PARAMS  = '/home/fesc/MM/Data/Params'
PATH_SAMPLES = '/home/fesc/MM/Data/Samples'
PATH_RESULTS = '/home/fesc/MM/Data/Results'

For the models specified, the code below runs the experiment for all given products (SKUs) and over parameter settings (e.g., cost parameters, horizon parameters, etc.). In total, we have 460 products (SKUs) with each 3 different cost parameter settings varying the critical ratio (CR) of holding and backlogging cost being {CR=0.50, CR=0.75, CR=0.90) and each 5 different lengths of the rolling look-ahead horizon tau being {1,2,3,4,5}.

## (a) Rolling Horizon Global Weighted SAA

In [ ]:
# Define paramaters
params = {
            
    # Sampling strategy
    'sampling': 'global',

    # Last sale_yearweek of training data
    'sale_yearweek': 114,

    # Length T of the test horizon
    'T': 13,

    # Lengths of rolling look-ahead horizons
    'tau': [1,2,3,4,5],

    # Cost param settings
    'cost_params': {

        1: {'CR': 0.50, 'K': 100, 'u': 0.5, 'h': 1, 'b': 1},
        2: {'CR': 0.75, 'K': 100, 'u': 0.5, 'h': 1, 'b': 3},
        3: {'CR': 0.90, 'K': 100, 'u': 0.5, 'h': 1, 'b': 9}

    },

    # Gurobi meta params
    'gurobi_params': {

        'LogToConsole': 1, 
        'Threads': 1, 
        'NonConvex': 2, 
        'PSDTol': 1e-3, # 0.1%
        'MIPGap': 1e-3, # 0.1%
        'NumericFocus': 0, 
        'obj_improvement': 1e-3, # 0.1%
        'obj_timeout_sec': 3*60, # 3 min
        'obj_timeout_max_sec': 10*60, # 10 min

    },
    
    'path': PATH_RESULTS+'/GwSAA',
    'model_name': 'GwSAA',
    
    'print_progress': False,
    'return_results': False
    
}

In [ ]:
# Set path
#os.mkdir(params['path'])
       
# Specify number of cores to use for parallel execution
n_jobs = 32

# Specify range of products (SKUs) to iterate over
SKU_range = range(1,460+1)

# Run for each product (SKU) in parallel
with tqdm_joblib(tqdm(desc='Progress', total=len(SKU_range))) as progress_bar:
    resultslog = Parallel(n_jobs=n_jobs)(delayed(run_experiments)(SKU, **params)
                                         for SKU in SKU_range)

## (b) Rolling Horizon Global Robust Weighted SAA

In [ ]:
# Define paramaters
params = {
            
    # Sampling strategy
    'sampling': 'global',

    # Robust uncertainty set threshold multiplier
    'e': None,

    # Last sale_yearweek of training data
    'sale_yearweek': 114,

    # Length T of the test horizon
    'T': 13,

    # Lengths of rolling look-ahead horizons
    'tau': [1,2,3,4,5],

    # Cost param settings
    'cost_params': {

        1: {'CR': 0.50, 'K': 100, 'u': 0.5, 'h': 1, 'b': 1},
        2: {'CR': 0.75, 'K': 100, 'u': 0.5, 'h': 1, 'b': 3},
        3: {'CR': 0.90, 'K': 100, 'u': 0.5, 'h': 1, 'b': 9}

    },

    # Gurobi meta params
    'gurobi_params': {

        'LogToConsole': 1, 
        'Threads': 1, 
        'NonConvex': 2, 
        'PSDTol': 1e-3, # 0.1%
        'MIPGap': 1e-3, # 0.1%
        'NumericFocus': 3, 
        'obj_improvement': 1e-3, # 0.1%
        'obj_timeout_sec': 3*60, # 3 min
        'obj_timeout_max_sec': 10*60, # 10 min

    },
    
    'path': PATH_RESULTS+'/GwSAAR',
    'model_name': 'GwSAAR',
    
    'print_progress': False,
    'return_results': False
    
}

In [ ]:
# Set path
#os.mkdir(params['path'])

# Specify number of cores to use for parallel execution
n_jobs = 32

# Specify range of products (SKUs) to iterate over
SKU_range = range(1,460+1)

# Uncertainty set
params['e'] = 1

# Run for each product (SKU) in parallel
with tqdm_joblib(tqdm(desc='Progress', total=len(SKU_range))) as progress_bar:
    resultslog = Parallel(n_jobs=n_jobs)(delayed(run_experiments)(SKU, **params)
                                         for SKU in SKU_range)

In [ ]:
# Set path
#os.mkdir(params['path'])

# Specify number of cores to use for parallel execution
n_jobs = 32

# Specify range of products (SKUs) to iterate over
SKU_range = range(1,460+1)

# Uncertainty set
params['e'] = 3

# Run for each product (SKU) in parallel
with tqdm_joblib(tqdm(desc='Progress', total=len(SKU_range))) as progress_bar:
    resultslog = Parallel(n_jobs=n_jobs)(delayed(run_experiments)(SKU, **params)
                                         for SKU in SKU_range)

In [ ]:
# Set path
#os.mkdir(params['path'])

# Specify number of cores to use for parallel execution
n_jobs = 32

# Specify range of products (SKUs) to iterate over
SKU_range = range(1,460+1)

# Uncertainty set
params['e'] = 6

# Run for each product (SKU) in parallel
with tqdm_joblib(tqdm(desc='Progress', total=len(SKU_range))) as progress_bar:
    resultslog = Parallel(n_jobs=n_jobs)(delayed(run_experiments)(SKU, **params)
                                         for SKU in SKU_range)

In [ ]:
# Set path
#os.mkdir(params['path'])

# Specify number of cores to use for parallel execution
n_jobs = 32

# Specify range of products (SKUs) to iterate over
SKU_range = range(1,460+1)

# Uncertainty set
params['e'] = 12

# Run for each product (SKU) in parallel
with tqdm_joblib(tqdm(desc='Progress', total=len(SKU_range))) as progress_bar:
    resultslog = Parallel(n_jobs=n_jobs)(delayed(run_experiments)(SKU, **params)
                                         for SKU in SKU_range)

## (c) Rolling Horizon Local Weighted SAA

## (d) Rolling Horizon Local Robust Weighted SAA

## (e) Rolling Horizon SAA

## (f) Ex-post optimal, deterministic model

# Aggregate all results